In [ ]:
import enum
from collections.abc import Callable
import didppy
import util

# Dual Bound を追加で設定する関数の型
type TypeBoundExprFunc = Callable[
    [list[str], didppy.Model, list[didppy.ElementVar]],
    didppy.IntExpr
]

In [ ]:
import marimo as mo
import nbformat

# DIDP を用いたモデル

考案者の解説記事: [動的計画法ベースの数理最適化ソルバDIDPPyで最短共通超配列問題を解く](https://zenn.dev/okaduki/articles/7f9a3f3c54bc98)

In [ ]:
def boundtable_scs2(s1: str, s2: str) -> list[list[int]]:
    len1, len2 = len(s1), len(s2)

    dp = [[len1 + len2 for _ in range(len2 + 1)] for _ in range(len1 + 1)]

    for i1 in range(len1 + 1):
        dp[i1][len2] = len1 - i1
    for i2 in range(len2 + 1):
        dp[len1][i2] = len2 - i2

    for i1 in range(len1 - 1, -1, -1):
        for i2 in range(len2 - 1, -1, -1):
            if s1[i1] == s2[i2]:
                dp[i1][i2] = dp[i1 + 1][i2 + 1] + 1
            else:
                dp[i1][i2] = min(dp[i1 + 1][i2], dp[i1][i2 + 1]) + 1

    return dp

In [ ]:
def boundexpr_scs2len(
    instance: list[str],
    dpmodel: didppy.Model,
    index_vars: list[didppy.ElementVar]
) -> didppy.IntExpr:
    exprs = []
    for idx1, (s1, index_var1) in enumerate(zip(instance, index_vars)):
        for idx2, (s2, index_var2) in enumerate(zip(instance, index_vars)):
            if idx2 >= idx1:
                continue
            table_idx1_idx2 = dpmodel.add_int_table(boundtable_scs2(s1, s2))
            exprs.append(table_idx1_idx2[index_var1, index_var2])

    bound = didppy.IntExpr(0)
    for expr in exprs:
        bound = didppy.max(bound, expr)

    return bound

In [ ]:
class Model:
    def __init__(
        self,
        instance: list[str],
        extra_bounds: list[TypeBoundExprFunc] | None = None,
        disable_default_bound: bool = False, 
    ):
        chars = sorted(list(set("".join(instance))))

        dpmodel = didppy.Model(maximize=False, float_cost=False)

        index_types = [dpmodel.add_object_type(number=len(s) + 1) for s in instance]
        index_vars = [
            dpmodel.add_element_var(object_type=index_type, target=0)
            for index_type in index_types
        ]

        instance_table = dpmodel.add_element_table(
            [
                [
                    chars.index(c) for c in s
                ] + [len(chars)]
                for s in instance
            ]
        )

        dpmodel.add_base_case(
            [
                index_var == len(s)
                for s, index_var in zip(instance, index_vars)
            ]
        )

        # 文字 char に従って進む
        for id_char, char in enumerate(chars):
            condition = didppy.Condition(False)
            for sidx, index_var in enumerate(index_vars):
                condition |= (instance_table[sidx, index_var] == id_char)
            trans = didppy.Transition(
                name=f"{char}",
                cost=1 + didppy.IntExpr.state_cost(),
                effects=[
                    (
                        index_var,
                        (
                            instance_table[sidx, index_var] == id_char
                        ).if_then_else(index_var + 1, index_var),
                    )
                    for sidx, index_var in enumerate(index_vars)
                ],
                preconditions=[condition],
            )
            dpmodel.add_transition(trans)

        # 残っている文字列から 2 つを選んで SCS を取って長さが最大のものを Dual Bound とする. 
        if not disable_default_bound:
            dpmodel.add_dual_bound(boundexpr_scs2len(instance, dpmodel, index_vars))

        # 追加の Dual Bound があれば. 
        if extra_bounds:
            for bound_func in extra_bounds:
                dpmodel.add_dual_bound(bound_func(instance, dpmodel, index_vars))

        self.instance = instance
        self.dpmodel = dpmodel
        self.dpsolver = None
        self.solution = None

    def solve(self, time_limit: int | None = 60, log: bool = False) -> "Model":
        self.dpsolver = didppy.CABS(self.dpmodel, threads=12, time_limit=time_limit, quiet=(not log))
        self.solution = self.dpsolver.search()
        return self

    def to_solution(self) -> str:
        return "".join([trans.name for trans in self.solution.transitions])

In [ ]:
def solve(instance: list[str], time_limit: int | None = 60, log: bool = False) -> str:
    model = Model(instance)
    model.solve(time_limit, log)
    return model.to_solution()

In [ ]:
instance_01 = util.parse("uniform_q26n004k015-025.txt")
model_01 = Model(instance_01)
solution_01 = model_01.solve().to_solution()

In [ ]:
_instance = instance_01
_model = model_01
_solution = solution_01

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 62) ---
 Sol: ultkcignyckuhojsimoqfevoazpplngbrddxxbcsvrvnnhstuqgpxzvxissbxf
str1: --tk--gn--kuh----m--------p--------x-------n-h-t-qg-xzvxis----
str2: -----i-------oj-i--qf--o----ln-b---xx-c-v-----s-uq-p--v-issbxf
str3: ul--ci-nyc---o-s--o---vo-zppl----------------------p----------
str4: -----ig--------------ev-az----gbrdd--bcsvrvnn-----g----------f

solution is feasible: True
solution is optimal: True
best bound: 62


In [ ]:
instance_02 = util.parse("uniform_q26n008k015-025.txt")
model_02 = Model(instance_02)
solution_02 = model_02.solve().to_solution()

In [ ]:
_instance = instance_02
_model = model_02
_solution = solution_02

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 98) ---
 Sol: iojiqfotkpbyplrgdenbxvawczxqgkbrufdrlchpmqvjgtdpfuivxnerzbycsodhutmsbopvroqgxzpvxinnsbsgwpdblhxfpe
str1: -------tk------g--n----------k--u-----h-m------p----xn---------h-t--------qgxz-vxi--s-------------
str2: iojiqfo------l----nbx-----x----------c----v-----------------s---u---------q---pv-i--s-s----b--xf--
str3: --------------------------------u---lc------------i--n----yc-o-----s-o-v-o---zp----------p--l---p-
str4: i--------------g-e---va--z--g-br--d-----------d----------b-cs----------vr------v--nn---g-------f--
str5: ---------p-yplr----------zx-----u----c-pmqv-gtd-fuiv-------c--d----sbo----------------------------
str6: ---------pb-----de---v------------d--c----v---dpf-------z---s-----

In [ ]:
instance_03 = util.parse("uniform_q26n016k015-025.txt")
model_03 = Model(instance_03)
solution_03 = model_03.solve().to_solution()

In [ ]:
_instance = instance_03
_model = model_03
_solution = solution_03

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 148) ---
  Sol: pyplrsxiutkoqjageiwnsbdzxqkefovudlchqnpazfgzbvxrdijmtqpvxerenlfygxqcvujzsheitodwauqbvmnctfeusompgvkbkzrhdivskjozxiqzupcgnwvdsrbbxhpaknwldizmgfoecsyp
str01: ---------tk----g---n------k----u---h---------------m--p-x---n------------h--t-----q-------------g---------------x--z------v-----x--------i-------s--
str02: -------i---o-j---i-------q--fo---l---n------b-x---------x----------cv---s--------uq------------p-v-------i-s----------------s-b-x------------f------
str03: ------

In [ ]:
instance_04 = util.parse("uniform_q05n010k010-010.txt")
model_04 = Model(instance_04)
solution_04 = model_04.solve().to_solution()

In [ ]:
_instance = instance_04
_model = model_04
_solution = solution_04

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 27) ---
  Sol: bbaeddcbeacdeecbdbeacbdcead
str01: ----d-cb--c---c-db--c--ce--
str02: b---dd-be---ee----e--bd----
str03: ------c--acdeec---e--b--e--
str04: --aedd-----d----d-e--bd---d
str05: --a---cbe---e-c----a-b-ce--
str06: bba----be------bd---cb---a-
str07: bbae-----a--e--b---a--d--a-
str08: ---e----e---eecbdbe-----e--
str09: ------c---cdee--d--a--dc--d
str10: b---d----a-----bdbea-----ad

solution is feasible: True
solution is optimal: True
best bound: 27


In [ ]:
instance_05 = util.parse("uniform_q05n050k010-010.txt")
model_05 = Model(instance_05)
solution_05 = model_05.solve().to_solution()

In [ ]:
_instance = instance_05
_model = model_05
_solution = solution_05

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 34) ---
  Sol: dcaebdaecbdeabcedacbedabcea

In [ ]:
instance_06 = util.parse("nucleotide_n010k010.txt")
model_06 = Model(instance_06)
solution_06 = model_06.solve().to_solution()

In [ ]:
_instance = instance_06
_model = model_06
_solution = solution_06

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 24) ---
  Sol: TACTACGCGTAGATCAGATCGTAC
str01: -A-T--G-G--GAT-A---CG---
str02: -A-TAC-C-T---TC----C---C
str03: --C-ACG---A-AT----T-G-A-
str04: TA--A-----A-ATC---T-GT--
str05: -A----G-GTA-A-CA-A----A-
str06: T--T-C-C-TAG----G-T---A-
str07: T--T--G--TAGATC---T-----
str08: T-----G-G--GA--AG-T--T-C
str09: T--T-C-C--A---CA-A-C-T--
str10: T-CTA-----A-A-C-GA----A-

solution is feasible: True
solution is optimal: True
best bound: 24


In [ ]:
instance_07 = util.parse("nucleotide_n050k050.txt")
model_07 = Model(instance_07)
solution_07 = model_07.solve().to_solution()

In [ ]:
_instance = instance_07
_model = model_07
_solution = solution_07

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

CT--T----TGCTA---A-G-A----A--T-----
str22: A---A--T--TAA---A--A-CAT--C--TC--A--A-T-A-CA--A--C-A---TAA-G-A---A--A---A--A--C----A--A--C---GCA-A-----A--A----AC--A--C-T-C-A--T-----
str23: A---A---A----CG-A--A-C-T-T---T---A--A---A--A---T-CT-G--T---GT--G-----G-C-T--G--T--CA-C-T-C---G--G-CTGC-A---TGCT---TA-G--TGC----------
str24: AT--A---A----C-TA--A---T-T-A--C-T--G--T---C-G--T--T-G---A-C--A-G-----G--A-CA--C--G-AG--T---A---A--CT-C--G--T-CTA--T---C-T--T-C-T--G--
str25: ATG-A-GT-GT--C--AC--G-A----A-T--T-C-AC-G-T-A-CA----A---T---G-A---ACT-G------GA-T-G-----T--T---CA--C-G-T-G---G--A---A----T---A----A---
str26: A--C-CGT-G----G-----GC--G--A---G--CG---G-T--G-A--C---CG----GT--GT-CT--TC--C----T---AG--TG----G--G--T-C---C---C-ACGT-----TG--A----A--R
str27: A---A---AG----GT--T----T---A-T---AC--CT--TC--C---C-AG-GTAAC--A---A--A--C--CA-AC---CA--A--CT-T------T-C--G-AT-CT-C-T-----TG-----------
str28: A-G----TAGT----T-C--GC----C--T-GT--G--TGA---GC-TG--A-C--AA---AC-T--TAGT-A---G--T-G-----T--T-TG-----TG--AG---G--

In [ ]:
instance_08 = util.parse("protein_n010k010.txt")
model_08 = Model(instance_08)
solution_08 = model_08.solve().to_solution()

In [ ]:
_instance = instance_08
_model = model_08
_solution = solution_08

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 44) ---
  Sol: MAEQSKPFTLRSLENYAQHVFDAICNSRPGFVKGTNEGALHDYQ
str01: MA-------L-S---Y--------C---P---KGT---------
str02: M--QS------SL-N-A------I----P--V------------
str03: M-----P--L-S---Y-QH-F------R----K-----------
str04: M-E----------E----HV-----N----------E--LHD--
str05: M---S---------N-----FDAI---R----------AL----
str06: M------F--R---N--Q-------NSR-------N-G------
str07: M------F-------YA-H---A-------F--G---G----Y-
str08: M---SK-FT-R----------------RP-------------YQ
str09: M---S--F-----------V--A------G-V--T---A----Q
str10: M-E-S----L---------V--------PGF----NE-------

solution is feasible: True
solution is optimal: False
best bound: 29


In [ ]:
instance_09 = util.parse("protein_n050k050.txt")
model_09 = Model(instance_09)
solution_09 = model_09.solve().to_solution()

In [ ]:
_instance = instance_09
_model = model_09
_solution = solution_09

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
    print(f"solution is optimal: {_model.solution.is_optimal}")
    print(f"best bound: {_model.solution.best_bound}")
else:
    print("--- Solution not found ---")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

-----D----------K-----------VF--------------------RS------S---------------V--L--------------H-------S----
str11: M---------------D---------S----------K-E-----T-I--------L---------------I-------E-----I-----------------I------------------P-----------K--------I--K--------S--------Y------------L----------L--------D-----------T----N---------I-S-------------------------P----K------------------S-----------------------YN---------D----F----------I---S----R-----N-K-----------N--------------------I-------------F---------V---I--N--------------L-------Y-----N--V-----S-----------T------------I---------------------
str12: M-----------------L-----L-S---G------K--------------------K-----K--------M--------------L-------LD---------------N-----------------------Y----E------T-----A-----------AA--R-------G-----------R---------G-----------------G---------------D----------E--------R-------------------R----R----------G--------------W------A---F-----------D-------R------------P----------A----------------I------

--------VF--------------------RS------S---------------V--L--------------H-------S----
str19: M------EA--I--------------------------I-----------------------------S------F--------A--------------G----I------------GI----------N-------Y---------K--K--------L--------------Q----------S----------------KL---------Q---------H-----------D----------------F----G-----------------R------VL--------K----------AL-----T-------------VT-A---------R------A----------L------PG--------------------------------Q--------------------------P------------KH-------------I------A-------------I------R----Q---------
str20: M-A------------S----------S---G---P----E-----------R-----A-E---------------------------H-Q--------------I-------------I--------------------------L-------P----E-----S-----------------------HL-----------------S----S-------------P------L--V--K--------HK-----------------L-----------L---------------------Y---------------------------------YW--------K---------L---T---G--------L----------P-------L---P--------D

-------VF--------------------RS------S---------------V--L--------------H-------S----
str29: M------------W-S----------------------I--------I--V-----L-K--------L----I-S-----------I--Q----P-L----L--------------L---V------------T----------------------S--L----------P-------L--------Y--------N--------P---------N--M----------------D-----------S---------------C-------C-------L--------I-------S-R---------I-T-----P------------------------------E-------L-----A-G----K---L------T-----------------------W--I-----------F------I--------------------------------------------------------------------
str30: M------E-------S--L-------------V-P---------G---------------------F--N----------E------------------------K------T--------H--------------V-------------------------------------Q---L------S---L---P-----------------V--L---------Q-----------V------------------R---D-V-----L--------------V---------------R------G-----------F-G---------D--S---------V----E------------E-------------F----------------L-------SE-----

-------------------N------D-----SD-T-----G--E----------D--------------L--------------V--------------D---------------
str39: M----F----V------FL-------------V-------L---------------L-------------------------P-----L---------V-----------------------S----S----Q-------C-----------V-----------------------N-L------------R------------------T------R---T--Q--------L-------------------P------------------P----S-----------------------Y--------T------------N--------S-F---T--R-----------GV---Y------------Y-----------P----------D----------K-----------VF--------------------RS------S---------------V--L--------------H-------S----
str40: M------E-------S--L-------------V-P---------G---------------------F--N----------E------------------------K------T--------H--------------V-------------------------------------Q---L------S---L---P-----------------V--L---------Q-----------V-----C-D------V---------------L--------------V---------------R------G-----------F-G---------D--S---------V----E------------E-------------

-P----------D----------K-----------VF--------------------RS------S---------------V--L--------------H------------
str45: M--------------S-------K-------D--------L---------V------A----R--------------------------Q-A----L---------M-----T--A--------R--------------M-------K-------A-----------------D--------F-------V------------------F------F----------------L------------------F--------V-----L---------------------W--K----------AL------S---------L----------------------------P---V-------P----------------T----R----C------Q---------I----D--------M--------A------K---------------K-------------L----------S--A--G----------
str46: M-A------------S--L-----L------------K------------------------------S-------------------L-----------------------T---L--------------F---K------------RT---------------------R-DQ-----------P------P-L----------AS-----------G-------S---------G-------------------G------A-----------------------I---------R------G--I-------------K--------H----------V-----I--I--VL----------------------

E---------L---------R----------R---N-------------Q-----------A-----L-N-K----------------N-----------------------L

solution is feasible: True
solution is optimal: False
best bound: 93
